# ANALYSIS OF METFORMIN CONTROL

This notebook contains an analysis of the effect of metformin on various tissues and organs as published in 


<img src="images/metformin-model-fits.png"
     alt="Markdown Monster icon" 
     width="800" height="1000"
     style="float: left; margin-right: 10px;" />

In [ ]:
import controlSBML as ctl

import control
import lmfit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
ctl.__version__

# Problem Statement

Metformin has proven effective at address type 2 diabetes. As such, there may be benefits if it is possible to maintain a constant dosage in physiological targets such as muscle. This is an exploratory study to investigate if the control of metformin in the blood (venus plasma) is sufficient to control metformin in muscle. This study considers a closed loop system in which: (a) metformin concentration is measured in muscle and (b) a controller can add metformin to the blood if its muscle concentration is low. We assume that there is no capability for removing metformin from the blood other than by physiological processes (e.g., intestinal absorption).

## Metformin Model

In [ ]:
url = "https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000001039.5?filename=Zake2021_Metformin%2BMice%2BIV.xml"
METFORMIN_CTLSB = ctl.ControlSBML(url, input_names=["mPlasmaVenous"], output_names=["mMuscle"])
#print(ctlsb.antimony)
METFORMIN_BUILDER = METFORMIN_CTLSB.makeSISOTransferFunctionBuilder(is_fixed_input_species=False)

In [ ]:
METFORMIN_CTLSB.roadrunner.simulate()
METFORMIN_CTLSB.roadrunner.plot()

# System Identification

In [ ]:
METFORMIN_STAIRCASE = ctl.Staircase(initial_value=0, final_value=4000, num_step=5)
response_ts = METFORMIN_BUILDER.makeStaircaseResponse(staircase=METFORMIN_STAIRCASE, end_time=50)
METFORMIN_BUILDER.plotStaircaseResponse(response_ts)

* The system is controllable in the region V1 $>4$, although the control decreases with larger V1 for both complexes.
* The operating region for V1 implies an operating region for pmTORC1 of 180 to 220 and for pmTORC2 0.7 to 1.0.

In [ ]:
fitter_result = METFORMIN_BUILDER.fitTransferFunction(num_numerator=2, num_denominator=3, 
                                                      staircase=METFORMIN_STAIRCASE, fit_start_time=10, start_time=0, end_time=50)
METFORMIN_BUILDER.plotFitTransferFunction(fitter_result)

# Testbed Construction

Here, we build a closed loop system where the open loop is the SBML model of the simulated system. Our has a PI controller.

In [ ]:
siso = ctl.SISOClosedLoopSystem(METFORMIN_CTLSB)
siso.makePIDClosedLoopSystem(kp=0.1, ki=0.01, is_nonnegative_output=True, closed_loop_outputs=["mMuscle", "mPlasmaVenous"])

In [ ]:
result_ts = siso.makeStepResponse(start_time=0, end_time=60, step_size=500)

In [ ]:
df = result_ts.copy()
times = np.array(df.index)/1000
plt.plot(times, df["mMuscle"], color="blue")
ax = plt.gca()
ax.plot(times, result_ts["reference"], linestyle="--", color = "blue")
ax2 = ax.twinx()
ax2.plot(times, result_ts["mPlasmaVenous"], color="red")
ax2.set_ylim([0, 100])
ax2.set_ylabel("mPlasmaVenous", color="red")
ax.set_ylabel("mMuscle", color="blue")

In [ ]:
min(result_ts["mPlasmaVenous"] )

# Closed Loop Design and Analysis